In [1]:
from keras.utils import np_utils
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import math
import cv2 as cv
import pandas as pd
import os

In [2]:
DIRECTORY = R'G:\pizza_not_pizza'

In [3]:
CATEGORIES = ['not_pizza']
IMG_SIZE = 150
data = []

for category in CATEGORIES:
    folder = os.path.join(DIRECTORY, category)
    label = CATEGORIES.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_arr = cv.imread(img_path)
        img_arr = cv.resize(img_arr, (IMG_SIZE, IMG_SIZE))
        data.append([img_arr, label])

In [4]:
train_data = data
original_X = []
original_Y = []

for features, label in train_data:
    original_X.append(features)
    original_Y.append(label)

original_X = np.array(original_X)
original_Y = np.array(original_Y)

original_X.shape

(983, 150, 150, 3)

In [5]:
# original_X = np.array(original_X)
# original_Y = np.array(original_Y)

In [6]:
# original_X.shape

In [7]:
X_gray=[]
for i in range(0, len(original_X)):
        X_gray.append(cv.cvtColor(original_X[i], cv.COLOR_BGR2GRAY))
X_gray=np.array(X_gray)
print(X_gray.shape)

(983, 150, 150)


In [8]:
# print(X_gray.shape)

In [9]:
CATEGORIES_1 = ['pizza']
IMG_SIZE = 150
data_1 = []

for category in CATEGORIES_1:
    folder = os.path.join(DIRECTORY, category)
    label = CATEGORIES_1.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_arr = cv.imread(img_path)
        img_arr = cv.resize(img_arr, (IMG_SIZE, IMG_SIZE))
        data_1.append([img_arr, label])

In [10]:
train_data_1 = data_1
original_X_1 = []
original_Y_1 = []
for features, label in train_data_1:
    original_X_1.append(features)
    original_Y_1.append(label)

original_X_1 = np.array(original_X_1)
original_Y_1 = np.array(original_Y_1)

original_X_1.shape

(983, 150, 150, 3)

In [11]:
# original_X_1 = np.array(original_X_1)
# original_Y_1 = np.array(original_Y_1)

In [12]:
# original_X_1.shape

In [13]:
X_gray_1=[]
for i in range(0, len(original_X_1)):
        X_gray_1.append(cv.cvtColor(original_X_1[i], cv.COLOR_BGR2GRAY))
X_gray_1=np.array(X_gray_1)

print(X_gray_1.shape)

(983, 150, 150)


In [14]:
# print(X_gray_1.shape)

In [15]:
def mean_(img):
    mean = 0
    for dimention_1 in range(0,img.shape[0]):
        for dimention_2 in range(0,img.shape[1]):
            mean = mean + img[dimention_1][dimention_2]
    mean = mean / img.size
    return mean

In [16]:
def median_(img):
    
    pixel_values = []
    for dimention_1 in range(0,img.shape[0]):
        for dimention_2 in range(0,img.shape[1]):
            pixel_values.append(img[dimention_1][dimention_2])

    pixel_values.sort()
    mid_index = math.floor(len(pixel_values)/2)
    median = pixel_values[mid_index]
    return median

In [17]:
def variance_(img, mean):
    variance=0
    for dimention_1 in range(0,img.shape[0]):
        for dimention_2 in range(0,img.shape[1]):
            variance= variance+ math.pow(img[dimention_1][dimention_2]-mean ,2)
    variance=variance/img.size
    return variance

In [18]:
def std_deviation_(variance):
    return math.sqrt(variance)

In [19]:
def skewness_(mean, meadian, std_deviation):
    return 3*((mean-median)/std_deviation)

In [20]:
def threshold_(img):
    ret,thresh1 = cv.threshold(img,127,255,cv.THRESH_BINARY)
    return thresh1

In [21]:
def hist_(img):
    hist = cv.calcHist([img],[0],None,[256],[0,256])
    return hist.flatten().mean()

In [22]:
def sobel_(img):
    sobelx = cv.Sobel(src=img, ddepth=cv.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
    sobely = cv.Sobel(src=img, ddepth=cv.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
    sobelxy = cv.Sobel(src=img, ddepth=cv.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection
    return (sobelx.flatten().mean(),sobely.flatten().mean(),sobelxy.flatten().mean())

In [23]:
def canny_(img):
    edges = cv.Canny(image=img, threshold1=100, threshold2=200) # Canny Edge Detection
    return edges.flatten().mean()

In [24]:
data0 = []
#generating class 0 features dataset
for test_image in X_gray:
    mean = mean_(test_image)
    median = median_(test_image)
    var = variance_(test_image, mean)
    std_deviation = std_deviation_(var)
    skewness = skewness_(mean, median, std_deviation)
    sobelx,sobely,sobelxy = sobel_(test_image)
    hist = hist_(test_image)
    canny = canny_(test_image)
#     print(mean, median, var, std_deviation, skewness , sobelx, sobely, sobelxy, hist, canny)
    data0.append({'mean':mean,
            'median':median,
            'var':var,
            'std_deviation':std_deviation,
            'skewness':skewness,
            'sobelx':sobelx,
            'sobely':sobely,
            'sobelxy':sobelxy,
            'hist':hist,
            'canny':canny,
            'class': 0})
    
df0 = pd.DataFrame(data = data0)
df0

,mean,median,var,std_deviation,skewness,sobelx,sobely,sobelxy,hist,canny,class
0,106.434267,115,2834.644168,53.241376,-0.482655,71.743600,34.991067,0.065822,87.890625,18.088000,0
1,106.261067,114,1982.922244,44.530015,-0.521374,-28.090267,72.552844,-0.021111,87.890625,17.408000,0
2,146.844356,192,7133.960841,84.462778,-1.603865,-7.432178,-5.144089,-0.271911,87.890625,21.238667,0
3,113.117422,124,3128.749323,55.935224,-0.583670,9.886267,4.965378,-0.044667,87.890625,32.334000,0
4,136.979067,119,4385.337651,66.221882,0.814492,-14.024356,47.949600,-0.150222,87.890625,34.396667,0
...,...,...,...,...,...,...,...,...,...,...,...
978,82.198133,67,3554.573099,59.620241,0.764747,-37.159111,-28.196267,-0.496711,87.890625,44.222667,0
979,93.955822,85,3508.789782,59.235038,0.453574,19.929956,114.321156,-0.005422,87.890625,27.494667,0
980,85.319022,77,4638.820447,68.108887,0.366429,-13.143644,-7.895378,0.246311,87.890625,48.393333,0
981,106.152933,91,2927.986611,54.110873,0.840105,-13.032000,56.552622,-0.213067,87.890625,34.793333,0


In [25]:
data1 = []
#generating class 1 features dataset
for test_image in X_gray_1:
    mean = mean_(test_image)
    median = median_(test_image)
    var = variance_(test_image, mean)
    std_deviation = std_deviation_(var)
    skewness = skewness_(mean, median, std_deviation)
    sobelx,sobely,sobelxy = sobel_(test_image)
    hist = hist_(test_image)
    canny = canny_(test_image)
#     print(mean, median, var, std_deviation, skewness , sobelx, sobely, sobelxy, hist, canny)
    data1.append({'mean':mean,
            'median':median,
            'var':var,
            'std_deviation':std_deviation,
            'skewness':skewness,
            'sobelx':sobelx,
            'sobely':sobely,
            'sobelxy':sobelxy,
            'hist':hist,
            'canny':canny,
            'class': 1})
    
df1 = pd.DataFrame(data = data1)
df1

,mean,median,var,std_deviation,skewness,sobelx,sobely,sobelxy,hist,canny,class
0,108.477067,98,3366.253830,58.019426,0.541736,30.441156,-62.653600,-0.252267,87.890625,68.328667,1
1,109.378400,113,1138.105080,33.735813,-0.322055,-18.513111,26.508222,0.033911,87.890625,41.514000,1
2,141.184311,161,3559.476563,59.661349,-0.996408,50.870000,67.627689,0.064756,87.890625,55.148000,1
3,94.706089,83,4994.690283,70.673123,0.496911,-26.869200,47.507333,-0.087422,87.890625,33.546667,1
4,93.428889,81,3277.298365,57.247693,0.651322,-29.008800,15.474222,0.103022,87.890625,35.416667,1
...,...,...,...,...,...,...,...,...,...,...,...
978,93.482000,91,2453.729943,49.535138,0.150318,-79.695733,41.055644,-0.804889,87.890625,65.529333,1
979,87.063467,77,2222.120150,47.139369,0.640450,13.422711,-10.413911,0.579067,87.890625,59.296000,1
980,118.435511,107,2606.402463,51.052938,0.671980,42.235644,-93.791467,-0.340000,87.890625,61.562667,1
981,101.450000,93,2592.189367,50.913548,0.497903,27.765111,-28.640400,0.221733,87.890625,63.761333,1


In [26]:
df0.to_csv('class0.csv', index=False)
df1.to_csv('class1.csv', index=False)

In [27]:
frames = [df0, df1]
df = pd.concat(frames)
df = df.sample(frac = 1) #shuffling data
df.to_csv('classes.csv', index=False)

In [28]:
!pip install scikit-learn

In [29]:
import pandas
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

df = pandas.read_csv("classes.csv")

train, test = train_test_split(df, test_size=0.2)

features = ['mean', 'median', 'var', 'std_deviation', 'skewness' , 'sobelx', 'sobely', 'sobelxy', 'hist', 'canny']


X = train[features]
y = train['class']

dtree = DecisionTreeClassifier()
dtree = dtree.fit(X, y)

In [30]:
x_test = test[features]
y_test = test['class']
dtree.score(x_test,y_test)

0.6700507614213198

In [31]:
from sklearn import svm
from sklearn.metrics import accuracy_score

ls = svm.SVC(kernel='linear')
ls.fit(X,y)


SVC(kernel='linear')

In [33]:
predicted = ls.predict(x_test)
acc = accuracy_score(y_test, predicted)
print(acc)

0.700507614213198


In [38]:
less_features = ['mean' , 'median', 'var', 'skewness', 'hist', 'canny']
X_less = train[less_features]
y = train['class']
x_test_less = test[less_features]
y_test = test['class']
ls01 = svm.SVC(kernel='linear')
ls01.fit(X_less,y)
predicted = ls01.predict(x_test_less)
acc = accuracy_score(y_test, predicted)
print(acc)

0.7309644670050761


In [44]:
two_features = ['skewness', 'canny']
X_2 = train[two_features]
y = train['class']
x_test_2 = test[two_features]
y_test = test['class']
ls02 = svm.SVC(kernel='linear')
ls02.fit(X_2,y)
predicted = ls02.predict(x_test_2)
acc = accuracy_score(y_test, predicted)
print(acc)

0.7131979695431472
